# Level 2 - Week 8 - 02 Sessions, Permissions, Audit

**Estimated time:** 60-90 minutes

## Learning Objectives

- Define session schema
- Specify permission rules
- Capture audit logs


## Overview

When you add ingestion/admin endpoints, you must think about:

- permissions
- auditing
- cost control

## Underlying theory: authn, authz, and least privilege

### Authentication vs authorization

- authentication (authn): who are you?
- authorization (authz): what are you allowed to do?

Many systems fail because they implement authn (a token) but forget authz (which endpoints that token can call).

### Least privilege

Admin endpoints have higher blast radius:

- `/ingest` can change the knowledge base
- `/eval` can spend tokens/cost and leak prompts in logs

So you want the minimal set of permissions needed for each role.

## Practice Steps

- Define roles (user/admin).
- Implement default-deny authz for admin routes.
- Draft an audit log entry (who/what/when/status).

### Sample code

Session schema and audit log entry.


In [ ]:
from datetime import datetime, timezone

SESSION_SCHEMA = {
    "session_id": "string",
    "user_id": "string",
    "role": "user|admin",
}


def make_audit_entry(request_id: str, actor: str, action: str, resource: str, status: str) -> dict:
    return {
        "request_id": request_id,
        "actor": actor,
        "action": action,
        "resource": resource,
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "status": status,
    }


audit = make_audit_entry("req-123", actor="admin", action="ingest", resource="data/doc.md", status="success")
print(SESSION_SCHEMA)
print(audit)

### Student fill-in

1. Implement `can_access(role, endpoint)` with **default deny** for admin endpoints.
2. Create an audit log entry shape that captures:

- `request_id`
- `actor` (user/admin)
- `action` (ingest/eval)
- `resource`
- `timestamp`
- `status`

In [ ]:
ADMIN_ENDPOINTS = {"/ingest", "/eval"}


def can_access(role: str, endpoint: str) -> bool:
    role = (role or "").strip()

    # Default allow: user endpoints
    if endpoint not in ADMIN_ENDPOINTS:
        return role in {"user", "admin"}

    # Default deny: admin endpoints
    return role == "admin"


print(can_access("user", "/chat"))
print(can_access("user", "/ingest"))
print(can_access("admin", "/ingest"))

## Self-check

- Are admin actions logged?
- Is default deny enforced for admin endpoints?
